<a href="https://colab.research.google.com/github/harshitadd/AI4BharatTranslation/blob/main/LaBSEAligner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers

In [2]:
!pip install scipy

In [ ]:
!pip install indic-nlp-library

In [4]:
import sys
from indicnlp.tokenize import indic_tokenize
from sentence_transformers import SentenceTransformer
import numpy as np
from scipy.spatial import distance
from transformers import AutoTokenizer, AutoModel

# tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/LaBSE")
# model = AutoModel.from_pretrained("sentence-transformers/LaBSE")
model = SentenceTransformer('LaBSE')



100%|██████████| 1.75G/1.75G [00:30<00:00, 58.0MB/s]


In [5]:
!pip install sentence-splitter

     |████████████████████████████████| 51kB 3.9MB/s 


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
with open('/content/drive/MyDrive/eng_2.txt','r') as file:
  content = file.read()

In [12]:
# Only for English
import re 
from sentence_splitter import SentenceSplitter, split_text_into_sentences
content = content.replace('\n',' ')
splitter = SentenceSplitter(language='en')
sentences = splitter.split(content)

# final = ''
# for s in sentences:
# 	final+=s
# 	final+='\n'
# sentence_count=len(sentences)
# print(sentence_count)


In [17]:
len(sentences)

813

In [19]:
temp = sentences[0:20]

In [15]:
def phrase_aligner(src, tgtp):
      '''
      This function is meant to align src phrases with best possible tgt phrase using LABSE model
      '''
      out = {}
      aligned_phrases = {}
      print("Performing phrase alignenment using LABSE")
      print(src,tgtp)
      src_phrases, tgt = src, tgtp
      
      for src_phrase in src_phrases:
          length_src_phrase = len(src_phrase.split())           
          tgt_token_list = split_tgt(length_src_phrase,tgt)
          embeddings_src_phrase, embeddings_tgt_tokens = generate_embeddings([src_phrase],tgt_token_list)
          alignments = get_target_sentence(embeddings_tgt_tokens, embeddings_src_phrase, length_src_phrase)
      
          if alignments is not None and alignments[2] is "MATCH":
              aligned_phrases[src_phrase] = tgt_token_list[alignments[0]]
          elif alignments is not None and alignments[2] is "NOMATCH": 
              print("No exact match found for:{} . Possible alignment {}".format(src_phrase,tgt_token_list[alignments[0]]))  
                      
      print("Aligned Phrases: {}".format(aligned_phrases))
      out = {"tgt":tgt,"src_phrases":src_phrases,"aligned_phrases":aligned_phrases}     

      return out

def split_tgt(length_src_phrase,tgt):
  tgt_token_list = list()
  tokenised_tgt = tgt.split()
  tgt_token_list = [tokenised_tgt[i:i+length_src_phrase] for i in range(len(tokenised_tgt)) if (i + length_src_phrase) <= len(tokenised_tgt)]
  tgt_token_list_plus = [tokenised_tgt[i:i+length_src_phrase+1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase+1) <= len(tokenised_tgt)]
  tgt_token_list_minus = [tokenised_tgt[i:i+length_src_phrase-1] for i in range(len(tokenised_tgt)) if (i + length_src_phrase-1) <= len(tokenised_tgt) and 
                          length_src_phrase != 1]
  tgt_token_list = tgt_token_list + tgt_token_list_plus + tgt_token_list_minus
  tgt_token_list = [" ".join(j) for j in tgt_token_list]
  return tgt_token_list
      
def generate_embeddings(input_1, input_2):
  '''
  Generate LABSE embeddings
  Note: Inputs are array of strings
  '''           
  embeddings_input_1 = model.encode(input_1,show_progress_bar=True)
  embeddings_input_2 = model.encode(input_2,show_progress_bar=True)    
  print("LABSE embedding generation finished")
  return embeddings_input_1, embeddings_input_2
  
def get_target_sentence(target_embeddings, source_embedding, length_src_phrase):
  '''
  Calculate cosine similarity using scipy distance method
  '''
  distances = distance.cdist(source_embedding, target_embeddings, "cosine")[0]
  min_index = np.argmin(distances)
  min_distance = 1 - distances[min_index]
  print("Match score: {}".format(min_distance))
  if min_distance >= 0.5:
      return min_index, min_distance, "MATCH"
  else:
      return min_index, min_distance, "NOMATCH"     
      


In [ ]:
print(content)

In [21]:
tgtp = '''ஆன்மாவின்‌ முன்‌ தலைவணங்கி, 2018-2019 ஆம்‌ ஆண்டிற்கான
நிதிநிலை அறிக்கையை இந்தப்‌ பேரவையின்‌ முன்‌ சமர்ப்பிக்க
விழைகிறேன்‌.

2. மக்களின்‌ உள்ளங்களில்‌ நீங்கா இடம்‌ பெற்று, அவர்களின்‌
பேரன்பைப்‌ பெற்ற மாபெரும்‌ தலைவியாக விளங்கிய புரட்சித்‌ தலைவி
அம்மா அவர்கள்‌ தொடங்கி வைத்த அனைத்து திட்டங்களையும்‌
தொடர்ந்து செயல்படுத்துவதில்‌ இந்த அரசு உறுதியாக உள்ளது.
தொலைநோக்குப்‌ பார்வையும்‌, மதிநுட்பமும்‌ வாய்ந்த புரட்சித்‌ தலைவி
அம்மா அவர்களின்‌ “அமைதி, வளம்‌, வளர்ச்சி! என்ற கொள்கை முழக்கம்‌,
இந்த மாநிலத்தை முன்னேற்றப்‌ பாதையில்‌ கொண்டு செல்வதற்கான
வழிகாட்டும்‌ அடிப்படை கொள்கையாக தொடர்ந்து விளங்கும்‌. பொது
சுகாதாரம்‌, கல்வி, சமூகப்‌ பாதுகாப்பு போன்ற துறைகளில்‌
செயல்படுத்தப்பட்டு வரும்‌ பல்வேறு திட்டங்கள்‌, நமது மாநிலத்தின்‌
மனிதவள மேம்பாட்டுக்‌ குறியீடுகளை உயர்த்தி, நாட்டிலேயே
சமூகநலத்தைப்‌ பேணிப்‌ போற்றும்‌ முன்மாதிரி மாநிலமாகத்‌ தமிழ்நாட்டை
உருவாக்கியுள்ளது என்பதை யாராலும்‌ மறுக்க முடியாது. கடல்‌ அலை
உயரும்போது அனைத்துப்‌ படகுகளும்‌ ஒருசேர உயர்வது போல்‌,
மாநிலத்தின்‌ துரிதமான பொருளாதார வளர்ச்சி, அனைத்துக்‌
குடும்பங்களின்‌ பொருளாதார நிலையையும்‌ ஒருசேர உயர்த்தும்‌ என்பதில்‌
ஐயமில்லை. எனினும்‌, சமுதாயத்தில்‌ அனைத்து மக்களின்‌ வாழ்வையும்‌
மேம்படுத்தி, ஏற்றத்தாழ்வுகளைக்‌ களைந்து, பொருளாதாரச்‌ சம
நிலையைக்‌ கொண்டு வருவதற்காக, பல்வேறு நலத்திட்டங்களை,
குறிப்பாக ஏழை எளிய மக்களின்‌ மேம்பாட்டிற்கான திட்டங்களை
இந்த அரசு தொடர்ந்து செயல்படுத்தும்‌. எனவே, பொருளாதார
மேம்பாட்டிற்கு பெரும்‌ ஊக்கம்‌ தரும்‌ அதே வேளையில்‌, நலத்திட்டப்‌
பணிகளும்‌ இந்த அரசின்‌ வளர்ச்சிக்‌ கொள்கையில்‌ தொடர்ந்து

'''

In [ ]:
# with open('/content/drive/MyDrive/tam_0_2.txt','r') as file:
#   content = file.read()
src = temp 
# tgtp = content
res = phrase_aligner(src, tgtp)

In [ ]:
print(res)

{'tgt': 'भारत का संविधान अप्रैल,  को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित किए गएहै । पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता है ।', 'src_phrases': ['This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made', ''], 'aligned_phrases': {'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u20

In [ ]:
print(res['aligned_phrases'])

{'This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time': 'को यथाविद्यमान संसद्\u200c द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्\u200c द्वारा, संविधान (एक सौ', ' All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition': 'है । संसद्\u200c द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित', 'The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made': 'पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता', '': ''}


In [ ]:
for key in res['aligned_phrases']:
  print(key)
  print(res['aligned_phrases'][key])
  

This edition of the Constitution of India reproduces the text of the Constitution of India as amended by Parliament from time to time
को यथाविद्यमान संसद्‌ द्वारा समयसमय पर यथा संशोधित भारत केसंविधान के पाठ को उद्धृत करता है । संसद्‌ द्वारा, संविधान (एक सौ
 All amendments made by the Parliament up to and including the Constitution (One Hundred and Third Amendment) Act, 2019 are incorporated in this edition
है । संसद्‌ द्वारा, संविधान (एक सौ तीनवां संशोधन) अधिनियम, तक और उसको सम्मिलित करते हुए, किए गए सभी संशोधन, इस संस्करण मैं सम्मिलित
The foot notes below the text indicate the Constitution Amendment Acts by which such amendments have been made
पाठ के नीचे दिए गए पाद टिप्पण, संविधान संशोधन अधिनियमों, जिसके द्वारा ऐसे संशोधन किएगए है, को उपदर्शित करता


